In [1]:
# Libraries
import tensorflow
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, ConcatDataset
from transformers import AutoTokenizer, BertTokenizer 

import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sn
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

2023-06-13 14:21:50.801557: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# Function untuk konversi waktu dari detik ke jam, menit, detik
def evl_time(t):
    min, sec= divmod(t, 60)
    hr, min = divmod(min, 60)
    return int(hr), int(min), int(sec)

# Function untuk menyimpan model
def save_model(model, name):
    torch.save(model.state_dict(), name)

# Function untuk load model
def load_model(model, path):
    model.load_state_dict(torch.load(path), strict=False)
    return model

In [3]:
# Mengubah dataset menjadi beberapa mini-batch
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True)

    segments_tensors = [s[2] for s in samples]
    segments_tensors = pad_sequence(segments_tensors, batch_first=True)

    label_ids = torch.stack([s[3] for s in samples])
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)

    return ids_tensors, segments_tensors, masks_tensors, label_ids

In [4]:
# Function untuk pelatihan model
def train_model_ABSA(loader, epochs, model_name):
    history = {'loss' : []}
    all_data = len(loader)
    
    # Pengulangan epoch
    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        
        # Pengulangan setiap mini-batch
        for data in loader:
            t0 = time.time()
            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            label_ids = label_ids.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            loss = model_ABSA(ids_tensors=ids_tensors, lable_tensors=label_ids, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            losses.append(loss.item())
            loss.backward()
            optimizer_ABSA.step()
            optimizer_ABSA.zero_grad()

            finish_data += 1
            current_times.append(round(time.time()-t0,3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
            print('epoch:', epoch+1, "/" , epochs," batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)

        history['loss'].append(np.mean(losses))
        save_model(model_ABSA, model_name)
    
    return history

# Function untuk pengujian model
def test_model_ABSA(loader):
    pred = []
    truth = []
    
    with torch.no_grad():
        # Pengulangan setiap mini-batch
        for data in loader:
            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ABSA(ids_tensors, None, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            
            _, predictions = torch.max(outputs, dim=1)

            pred += list([int(i) for i in predictions])
            truth += list([int(i) for i in label_ids])

    return truth, pred

In [5]:
# Function untuk menampilkan confusion matrix
def draw_c_matrix(c_matrix, title="test"):
    aix = []
    
    for y in range(len(c_matrix)):
        aix.append(y)
    df_cm = pd.DataFrame(c_matrix, aix, aix)
    sn.heatmap(df_cm, annot=True, fmt='g', cmap='Blues')
    plt.ylabel("prediction")
    plt.xlabel("ground truth")
    plt.title(title)
    plt.show()

In [6]:
# Menentukan device yang akan digunakan untuk melakukan komputasi
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

print("Tensorflow\t: ", tensorflow.__version__)
print("Torch\t\t: ", torch.__version__)
print("Device\t\t: ", DEVICE)
print("GPU\t\t: ", torch.cuda.get_device_name())
print("CUDA\t\t: ", torch.version.cuda)

Tensorflow	:  2.4.1
Torch		:  1.1.0
Device		:  cuda:1
GPU		:  Tesla T4
CUDA		:  9.0.176


In [7]:
# Inisialisasi hyperparameter
bs = 16
lr = 0.00002
epoch = 25

In [8]:
# Inisialisasi pre-trained model BERT
from ABSA_SentimentMultiEmiten.model.bert import bert_ABSA
from ABSA_SentimentMultiEmiten.data.dataset import dataset_ABSA

model_name = "bert-multilingual-2.pkl"
pretrained_model_name = "bert-base-multilingual-uncased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
model_ABSA = bert_ABSA(pretrained_model_name)
model_ABSA.to(DEVICE)
optimizer_ABSA = torch.optim.Adam(model_ABSA.parameters(), lr=lr)

In [9]:
# Inisialisasi dataset
emiten_train_ds = dataset_ABSA(pd.read_csv("data_experiment/data_balance_experiment_training.csv"), tokenizer)
emiten_test_ds = dataset_ABSA(pd.read_csv("data_experiment/data_balance_experiment_testing.csv"), tokenizer)

train_ds = ConcatDataset([emiten_train_ds])
test_ds = ConcatDataset([emiten_test_ds])

# Mengubah dataset menjadi mini-batch
train_loader = DataLoader(train_ds, batch_size=bs, collate_fn=create_mini_batch, shuffle = True)
test_loader = DataLoader(test_ds, batch_size=bs, collate_fn=create_mini_batch, shuffle = True)

In [10]:
# Proses training model
history = %time train_model_ABSA(train_loader, epoch, model_name)

epoch: 1 / 25  batch: 1 / 612  loss: 1.1072043180465698  hr: 3  min: 11  sec: 44
epoch: 1 / 25  batch: 2 / 612  loss: 1.1296067237854004  hr: 3  min: 19  sec: 53
epoch: 1 / 25  batch: 3 / 612  loss: 1.1214791536331177  hr: 2  min: 46  sec: 54
epoch: 1 / 25  batch: 4 / 612  loss: 1.1297825276851654  hr: 2  min: 20  sec: 20
epoch: 1 / 25  batch: 5 / 612  loss: 1.1318218469619752  hr: 2  min: 26  sec: 7
epoch: 1 / 25  batch: 6 / 612  loss: 1.1268025437990825  hr: 2  min: 31  sec: 39
epoch: 1 / 25  batch: 7 / 612  loss: 1.1206428493772234  hr: 2  min: 34  sec: 40
epoch: 1 / 25  batch: 8 / 612  loss: 1.1160664707422256  hr: 2  min: 36  sec: 10
epoch: 1 / 25  batch: 9 / 612  loss: 1.1089333030912611  hr: 2  min: 39  sec: 6
epoch: 1 / 25  batch: 10 / 612  loss: 1.104271137714386  hr: 2  min: 45  sec: 56
epoch: 1 / 25  batch: 11 / 612  loss: 1.0976235649802468  hr: 2  min: 46  sec: 45
epoch: 1 / 25  batch: 12 / 612  loss: 1.0906620621681213  hr: 2  min: 48  sec: 7
epoch: 1 / 25  batch: 13 / 61

epoch: 1 / 25  batch: 102 / 612  loss: 0.9317616183383792  hr: 2  min: 47  sec: 27
epoch: 1 / 25  batch: 103 / 612  loss: 0.9311071310228515  hr: 2  min: 47  sec: 31
epoch: 1 / 25  batch: 104 / 612  loss: 0.9286283501065694  hr: 2  min: 47  sec: 37
epoch: 1 / 25  batch: 105 / 612  loss: 0.9252976213182722  hr: 2  min: 47  sec: 50
epoch: 1 / 25  batch: 106 / 612  loss: 0.9245086829617338  hr: 2  min: 47  sec: 55
epoch: 1 / 25  batch: 107 / 612  loss: 0.9215673191525112  hr: 2  min: 48  sec: 1
epoch: 1 / 25  batch: 108 / 612  loss: 0.9187990874052048  hr: 2  min: 48  sec: 41
epoch: 1 / 25  batch: 109 / 612  loss: 0.9152349507043122  hr: 2  min: 48  sec: 44
epoch: 1 / 25  batch: 110 / 612  loss: 0.9122342109680176  hr: 2  min: 47  sec: 50
epoch: 1 / 25  batch: 111 / 612  loss: 0.9097852373982335  hr: 2  min: 48  sec: 0
epoch: 1 / 25  batch: 112 / 612  loss: 0.9061323308518955  hr: 2  min: 47  sec: 23
epoch: 1 / 25  batch: 113 / 612  loss: 0.903911044112349  hr: 2  min: 47  sec: 27
epoch: 

epoch: 1 / 25  batch: 202 / 612  loss: 0.7931379124817282  hr: 2  min: 49  sec: 44
epoch: 1 / 25  batch: 203 / 612  loss: 0.7912466054039048  hr: 2  min: 50  sec: 3
epoch: 1 / 25  batch: 204 / 612  loss: 0.7896889852104234  hr: 2  min: 50  sec: 19
epoch: 1 / 25  batch: 205 / 612  loss: 0.7891624934062725  hr: 2  min: 50  sec: 38
epoch: 1 / 25  batch: 206 / 612  loss: 0.7882506414090545  hr: 2  min: 50  sec: 39
epoch: 1 / 25  batch: 207 / 612  loss: 0.7884824181668424  hr: 2  min: 50  sec: 43
epoch: 1 / 25  batch: 208 / 612  loss: 0.787641763758774  hr: 2  min: 51  sec: 2
epoch: 1 / 25  batch: 209 / 612  loss: 0.7871945224452818  hr: 2  min: 51  sec: 20
epoch: 1 / 25  batch: 210 / 612  loss: 0.7858004809135483  hr: 2  min: 51  sec: 22
epoch: 1 / 25  batch: 211 / 612  loss: 0.7842966635221553  hr: 2  min: 51  sec: 26
epoch: 1 / 25  batch: 212 / 612  loss: 0.7826811070290377  hr: 2  min: 51  sec: 29
epoch: 1 / 25  batch: 213 / 612  loss: 0.7816211204153831  hr: 2  min: 51  sec: 0
epoch: 1

epoch: 1 / 25  batch: 302 / 612  loss: 0.7118490120235658  hr: 2  min: 53  sec: 0
epoch: 1 / 25  batch: 303 / 612  loss: 0.7116316359231968  hr: 2  min: 53  sec: 6
epoch: 1 / 25  batch: 304 / 612  loss: 0.7115197123861626  hr: 2  min: 53  sec: 18
epoch: 1 / 25  batch: 305 / 612  loss: 0.7102126386321959  hr: 2  min: 53  sec: 29
epoch: 1 / 25  batch: 306 / 612  loss: 0.7096857425628924  hr: 2  min: 53  sec: 41
epoch: 1 / 25  batch: 307 / 612  loss: 0.7093648771703825  hr: 2  min: 53  sec: 40
epoch: 1 / 25  batch: 308 / 612  loss: 0.709149893131349  hr: 2  min: 53  sec: 43
epoch: 1 / 25  batch: 309 / 612  loss: 0.708846268433969  hr: 2  min: 53  sec: 55
epoch: 1 / 25  batch: 310 / 612  loss: 0.707877954456114  hr: 2  min: 54  sec: 8
epoch: 1 / 25  batch: 311 / 612  loss: 0.7064330840417427  hr: 2  min: 53  sec: 47
epoch: 1 / 25  batch: 312 / 612  loss: 0.7062096498333491  hr: 2  min: 53  sec: 48
epoch: 1 / 25  batch: 313 / 612  loss: 0.7059161449773624  hr: 2  min: 53  sec: 50
epoch: 1 /

epoch: 1 / 25  batch: 402 / 612  loss: 0.657809142190129  hr: 2  min: 54  sec: 24
epoch: 1 / 25  batch: 403 / 612  loss: 0.6577677737468526  hr: 2  min: 54  sec: 34
epoch: 1 / 25  batch: 404 / 612  loss: 0.6572529399527771  hr: 2  min: 54  sec: 41
epoch: 1 / 25  batch: 405 / 612  loss: 0.656596205447927  hr: 2  min: 54  sec: 51
epoch: 1 / 25  batch: 406 / 612  loss: 0.655887463960448  hr: 2  min: 54  sec: 50
epoch: 1 / 25  batch: 407 / 612  loss: 0.6562867496684377  hr: 2  min: 54  sec: 37
epoch: 1 / 25  batch: 408 / 612  loss: 0.6552636894744401  hr: 2  min: 54  sec: 44
epoch: 1 / 25  batch: 409 / 612  loss: 0.654099171712521  hr: 2  min: 54  sec: 26
epoch: 1 / 25  batch: 410 / 612  loss: 0.6532114454158923  hr: 2  min: 54  sec: 12
epoch: 1 / 25  batch: 411 / 612  loss: 0.6520928348212927  hr: 2  min: 54  sec: 1
epoch: 1 / 25  batch: 412 / 612  loss: 0.6514398188700954  hr: 2  min: 54  sec: 2
epoch: 1 / 25  batch: 413 / 612  loss: 0.6508894619728116  hr: 2  min: 53  sec: 58
epoch: 1 /

...

epoch: 25 / 25  batch: 185 / 612  loss: 0.03381373370821412  hr: 0  min: 5  sec: 4
epoch: 25 / 25  batch: 186 / 612  loss: 0.033727973258943965  hr: 0  min: 5  sec: 3
epoch: 25 / 25  batch: 187 / 612  loss: 0.03355115177956494  hr: 0  min: 5  sec: 3
epoch: 25 / 25  batch: 188 / 612  loss: 0.03337632947938239  hr: 0  min: 5  sec: 2
epoch: 25 / 25  batch: 189 / 612  loss: 0.033208517288720166  hr: 0  min: 5  sec: 1
epoch: 25 / 25  batch: 190 / 612  loss: 0.033307609432622  hr: 0  min: 5  sec: 1
epoch: 25 / 25  batch: 191 / 612  loss: 0.03314844639825571  hr: 0  min: 5  sec: 0
epoch: 25 / 25  batch: 192 / 612  loss: 0.03301622881554067  hr: 0  min: 4  sec: 59
epoch: 25 / 25  batch: 193 / 612  loss: 0.033015731471190184  hr: 0  min: 4  sec: 59
epoch: 25 / 25  batch: 194 / 612  loss: 0.03314285890496883  hr: 0  min: 4  sec: 59
epoch: 25 / 25  batch: 195 / 612  loss: 0.033524057612969324  hr: 0  min: 4  sec: 58
epoch: 25 / 25  batch: 196 / 612  loss: 0.03336935402939514  hr: 0  min: 4  sec: 

epoch: 25 / 25  batch: 283 / 612  loss: 0.037548359575625447  hr: 0  min: 3  sec: 56
epoch: 25 / 25  batch: 284 / 612  loss: 0.03753440354911374  hr: 0  min: 3  sec: 55
epoch: 25 / 25  batch: 285 / 612  loss: 0.03741011734594379  hr: 0  min: 3  sec: 54
epoch: 25 / 25  batch: 286 / 612  loss: 0.03728549838899733  hr: 0  min: 3  sec: 54
epoch: 25 / 25  batch: 287 / 612  loss: 0.03716073125496974  hr: 0  min: 3  sec: 53
epoch: 25 / 25  batch: 288 / 612  loss: 0.03704153523883886  hr: 0  min: 3  sec: 52
epoch: 25 / 25  batch: 289 / 612  loss: 0.037079743138646586  hr: 0  min: 3  sec: 51
epoch: 25 / 25  batch: 290 / 612  loss: 0.03720513908513661  hr: 0  min: 3  sec: 51
epoch: 25 / 25  batch: 291 / 612  loss: 0.03721908643790537  hr: 0  min: 3  sec: 50
epoch: 25 / 25  batch: 292 / 612  loss: 0.037107042330380986  hr: 0  min: 3  sec: 49
epoch: 25 / 25  batch: 293 / 612  loss: 0.0371246861465555  hr: 0  min: 3  sec: 48
epoch: 25 / 25  batch: 294 / 612  loss: 0.03700547718576023  hr: 0  min: 3

epoch: 25 / 25  batch: 381 / 612  loss: 0.038512757942667154  hr: 0  min: 2  sec: 46
epoch: 25 / 25  batch: 382 / 612  loss: 0.0384191494162014  hr: 0  min: 2  sec: 45
epoch: 25 / 25  batch: 383 / 612  loss: 0.03832376045089789  hr: 0  min: 2  sec: 45
epoch: 25 / 25  batch: 384 / 612  loss: 0.038366101284433775  hr: 0  min: 2  sec: 44
epoch: 25 / 25  batch: 385 / 612  loss: 0.03827510126805925  hr: 0  min: 2  sec: 43
epoch: 25 / 25  batch: 386 / 612  loss: 0.03820814650718103  hr: 0  min: 2  sec: 42
epoch: 25 / 25  batch: 387 / 612  loss: 0.03822592777760762  hr: 0  min: 2  sec: 42
epoch: 25 / 25  batch: 388 / 612  loss: 0.038131734941961225  hr: 0  min: 2  sec: 41
epoch: 25 / 25  batch: 389 / 612  loss: 0.03807898623279863  hr: 0  min: 2  sec: 40
epoch: 25 / 25  batch: 390 / 612  loss: 0.03798606566702708  hr: 0  min: 2  sec: 40
epoch: 25 / 25  batch: 391 / 612  loss: 0.037897376253095735  hr: 0  min: 2  sec: 39
epoch: 25 / 25  batch: 392 / 612  loss: 0.03806743482888049  hr: 0  min: 

epoch: 25 / 25  batch: 479 / 612  loss: 0.03882590793306494  hr: 0  min: 1  sec: 36
epoch: 25 / 25  batch: 480 / 612  loss: 0.03874793218759199  hr: 0  min: 1  sec: 35
epoch: 25 / 25  batch: 481 / 612  loss: 0.038708099100173386  hr: 0  min: 1  sec: 34
epoch: 25 / 25  batch: 482 / 612  loss: 0.038755837735175096  hr: 0  min: 1  sec: 33
epoch: 25 / 25  batch: 483 / 612  loss: 0.03883690291371652  hr: 0  min: 1  sec: 33
epoch: 25 / 25  batch: 484 / 612  loss: 0.03961551782082428  hr: 0  min: 1  sec: 32
epoch: 25 / 25  batch: 485 / 612  loss: 0.039560459584919445  hr: 0  min: 1  sec: 31
epoch: 25 / 25  batch: 486 / 612  loss: 0.03948554241792165  hr: 0  min: 1  sec: 31
epoch: 25 / 25  batch: 487 / 612  loss: 0.03956569717466464  hr: 0  min: 1  sec: 30
epoch: 25 / 25  batch: 488 / 612  loss: 0.03972032437192612  hr: 0  min: 1  sec: 29
epoch: 25 / 25  batch: 489 / 612  loss: 0.03968209695596636  hr: 0  min: 1  sec: 28
epoch: 25 / 25  batch: 490 / 612  loss: 0.039737153600673286  hr: 0  min:

epoch: 25 / 25  batch: 577 / 612  loss: 0.03852816775725247  hr: 0  min: 0  sec: 25
epoch: 25 / 25  batch: 578 / 612  loss: 0.03847273805252821  hr: 0  min: 0  sec: 24
epoch: 25 / 25  batch: 579 / 612  loss: 0.038407145875517364  hr: 0  min: 0  sec: 23
epoch: 25 / 25  batch: 580 / 612  loss: 0.03834184343958723  hr: 0  min: 0  sec: 22
epoch: 25 / 25  batch: 581 / 612  loss: 0.03827725956136306  hr: 0  min: 0  sec: 22
epoch: 25 / 25  batch: 582 / 612  loss: 0.038406466572350244  hr: 0  min: 0  sec: 21
epoch: 25 / 25  batch: 583 / 612  loss: 0.038341511762980535  hr: 0  min: 0  sec: 20
epoch: 25 / 25  batch: 584 / 612  loss: 0.038476941354368646  hr: 0  min: 0  sec: 20
epoch: 25 / 25  batch: 585 / 612  loss: 0.03841291287770638  hr: 0  min: 0  sec: 19
epoch: 25 / 25  batch: 586 / 612  loss: 0.03841330946356363  hr: 0  min: 0  sec: 18
epoch: 25 / 25  batch: 587 / 612  loss: 0.0384211649251959  hr: 0  min: 0  sec: 17
epoch: 25 / 25  batch: 588 / 612  loss: 0.03835889123076079  hr: 0  min: 

In [11]:
# Menampilkan nilai loss
history_df = pd.DataFrame(history)
history_df

,loss
0,0.574704
1,0.296443
2,0.192235
3,0.139346
4,0.118551
5,0.094920
6,0.074959
7,0.080327
8,0.080622
9,0.059170


In [12]:
model_ABSA = load_model(model_ABSA, model_name)

In [13]:
# Classification report
x, y = test_model_ABSA(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95       806
           1       0.87      0.90      0.89       808
           2       0.90      0.86      0.88       799

    accuracy                           0.91      2413
   macro avg       0.91      0.90      0.90      2413
weighted avg       0.91      0.91      0.90      2413

